In [184]:
import pandas as pd
import numpy as np

from keras.utils.np_utils import to_categorical

In [185]:
df = pd.read_csv("C:\\MyWork\\MyLearning\\ML\\Files\\DataSet\\uci-news-aggregator.csv",usecols=['TITLE','CATEGORY'])

In [186]:
df.head()

,TITLE,CATEGORY
0,"Fed official says weak data caused by weather,...",b
1,Fed's Charles Plosser sees high bar for change...,b
2,US open: Stocks fall after Fed official hints ...,b
3,"Fed risks falling 'behind the curve', Charles ...",b
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,b


In [187]:
# Shuffle the DataFrame

df = df.reindex(np.random.permutation(df.index))

In [188]:
df = df[:1000]

In [189]:
df.CATEGORY.value_counts()

e    401
t    265
b    241
m     93
Name: CATEGORY, dtype: int64

In [190]:
set(df['CATEGORY'])

{'b', 'e', 'm', 't'}

In [191]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
Y = le.fit_transform(df['CATEGORY'])

In [192]:
from sklearn.preprocessing import OneHotEncoder

In [193]:
ohe = OneHotEncoder()
y = ohe.fit_transform(df[['CATEGORY']]).toarray()

y

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [194]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import sequence
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = 8000,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)

tokenizer.fit_on_texts(df['TITLE'].values)
sequence = tokenizer.texts_to_sequences(df['TITLE'].values)
word_index = tokenizer.word_index

In [195]:
X = pad_sequences(sequence,maxlen=130)

In [205]:
from keras.models import Sequential
from keras.layers import Embedding,Dense,LSTM,SpatialDropout1D
from keras.callbacks import EarlyStopping

In [201]:
model = Sequential()
model.add(Embedding(8000,128,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.7))
model.add(LSTM(64,dropout=0.7,recurrent_dropout=0.7))
model.add(Dense(4,activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])



In [203]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 130, 128)          1024000   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 130, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 1,073,668
Trainable params: 1,073,668
Non-trainable params: 0
_________________________________________________________________


In [209]:
history = model.fit(X, y, epochs=30, batch_size=256,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=7, min_delta=0.0001)])

Train on 800 samples, validate on 200 samples
Epoch 1/30
800/800 [==============================] - 4s 5ms/step - loss: 0.9065 - acc: 0.7088 - val_loss: 1.1003 - val_acc: 0.5900
Epoch 2/30
800/800 [==============================] - 3s 4ms/step - loss: 0.8559 - acc: 0.7588 - val_loss: 1.0748 - val_acc: 0.6250
Epoch 3/30
800/800 [==============================] - 3s 4ms/step - loss: 0.8249 - acc: 0.7625 - val_loss: 1.0519 - val_acc: 0.6250
Epoch 4/30
800/800 [==============================] - 3s 4ms/step - loss: 0.7694 - acc: 0.7800 - val_loss: 1.0290 - val_acc: 0.6400
Epoch 5/30
800/800 [==============================] - 3s 4ms/step - loss: 0.7391 - acc: 0.8087 - val_loss: 1.0075 - val_acc: 0.6400
Epoch 6/30
800/800 [==============================] - 3s 4ms/step - loss: 0.6968 - acc: 0.8112 - val_loss: 0.9916 - val_acc: 0.6450
Epoch 7/30
800/800 [==============================] - 3s 4ms/step - loss: 0.6551 - acc: 0.8213 - val_loss: 0.9801 - val_acc: 0.6550
Epoch 8/30
800/800 [==========

In [213]:
txt = ["Regular fast food eating linked to fertility issues in women"]
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=130)

padded

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0, 2104,  413, 1406,  435,    1, 3231,    2, 1005]])

In [215]:
pred = model.predict(padded)

pred

array([[0.20239931, 0.09898496, 0.55342865, 0.145187  ]], dtype=float32)

In [216]:
labels = ['entertainment', 'bussiness', 'science/tech', 'health']
print(pred, labels[np.argmax(pred)])

[[0.20239931 0.09898496 0.55342865 0.145187  ]] science/tech


In [ ]:
ohe.